In [1]:
# !pip install datasets
import pandas as pd
from datasets import load_dataset
training_data1 = load_dataset('mc_taco', split="validation")
updated_dataset_test = [x for x in training_data1  if (x['label']!=0)]
datas = list()
prev = ''
c=0
lst=[]
file_path = '/content/drive/MyDrive/Llama train McTACO test McTACO final.csv'
df = pd.read_csv(file_path)
for temp in updated_dataset_test:
    if(temp['question']==prev):
      continue
    lst.append(temp['answer'])
    c+=1
    if(c==250):
      break
    prev=temp['question']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for mc_taco contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mc_taco
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating test split:   0%|          | 0/9442 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3783 [00:00<?, ? examples/s]

In [4]:
import pandas as pd
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import en_core_web_sm

nlp = en_core_web_sm.load()

def get_embedding(sentence):
    tokens = nlp(sentence)
    return np.mean([token.vector for token in tokens], axis=0)

def calculate_cosine_similarity(reference_sentence, predicted_sentence):
    reference_embedding = get_embedding(reference_sentence)
    predicted_embedding = get_embedding(predicted_sentence)

    reference_embedding = reference_embedding.reshape(1, -1)
    predicted_embedding = predicted_embedding.reshape(1, -1)

    similarity = cosine_similarity(reference_embedding, predicted_embedding)[0, 0]

    return similarity

file_path = '/content/drive/MyDrive/Llama train McTACO test McTACO final.csv'
df = pd.read_csv(file_path)
fscores=[]
edscores=[]
tscores=[]
sscores=[]
orderscores=[]
cosine_similarity_scores = []
ans=0
for index, row in df.iterrows():
    reference_question = lst[ans]
    ans+=1
    if(ans==250):
      break
    predicted_question = row['answer'].lstrip()

    similarity_score = calculate_cosine_similarity(reference_question, predicted_question)
    # if(ans<=10):
    #   print(row['context'])
    #   print(row['question'])
    #   print(reference_question)
    #   print(predicted_question)
    #   print(similarity_score)
    if(row['category']=='Frequency'):
      if(len(fscores)<30):
        fscores.append(similarity_score)
    elif(row['category']=='Event Duration'):
      if(len(edscores)<30):
        edscores.append(similarity_score)
    elif(row['category']=='Typical Time'):
      if(len(tscores)<30):
        tscores.append(similarity_score)
    elif(row['category']=='Stationarity'):
      if(len(sscores)<30):
        sscores.append(similarity_score)
    else:
      if(len(orderscores)<30):
        orderscores.append(similarity_score)

# df['cosine_similarity'] = cosine_similarity_scores
# print(sum(cosine_similarity_scores)/len(cosine_similarity_scores))
# print(df)


In [3]:
print(sum(fscores)/len(fscores))
print(sum(sscores)/len(sscores))
print(sum(edscores)/len(edscores))
print(sum(orderscores)/len(orderscores))
print(sum(tscores)/len(tscores))

0.3702583409845829
0.4408448196947575
0.556715622295936
0.38428685689965886
0.5266067492465178
